# Лекция 11

## Итерационные методы для поиска собственных значений

## На прошлой лекции

- Методы MINRES, BiCG и BiCGStab 
- Предобуславливатели: метод Якоби, Гаусса-Зейделя и SOR($\omega$)
- Неполное LU разложение и его варианты

## Частичная задача на собственные значения

- Напомним, что для поиска собственных значений матрицы $N\times N$ можно использовать например QR алгоритм.

- Однако в некоторых приложениях матрицы настолько большие, что мы даже не можем их хранить в памяти 

- Обычно такие матрицы даны в виде **чёрного ящика**, который позволяет только умножить эту матрицу на вектор. Далее рассмотрим именно этот случай.

- Лучшее, что мы можем сделать это решить частичную задачу на собственные значения, то есть

    - Найти $k\ll N$ наименьших или наибольших собственных значений (и собственных векторов, если необходимо)
    - Найти $k\ll N$ собственных значений, ближайших к заданному числу $\sigma$

- Для простоты рассмотрим случай нормальной матрицы, которая имеет ортонормированный базис из собственных векторов. 



### Степенной метод и его аналоги

#### Степенной метод: напоминание

- Простейший метод для поиска максимального по модулю собственного значения – это **степенной метод**

$$ x_{i+1} = \frac{Ax_{i}}{\|Ax_{i}\|_2}. $$

- Сходимость линейная с коэффициентом $q = \left|\frac{\lambda_1}{\lambda_2}\right|$.

#### Метод обратной итерации: напоминание

- Для поиска наименьшего собственного значения можно запустить степенной метод для матрицы $A^{-1}$:

$$x_{i+1} = \frac{A^{-1}x_{i}}{\|A^{-1}x_{i}\|}.$$

- Для ускорения сходимости может быть использована стратегия <font color='red'>shift-and-invert</font>:

$$x_{i+1} = \frac{(A-\sigma I)^{-1}x_{i}}{\|(A-\sigma I)^{-1}x_{i}\|},$$

где $\sigma$ должна лежать близко от целевого собственного значения.

#### Метод Релея

- Для получения сверхлинейной сходимости можно воспользоваться адаптивными сдвигами:

$$x_{i+1} = \frac{(A-R(x_i) I)^{-1}x_{i}}{\|(A-R(x_i) I)^{-1}x_{i}\|},$$

где $R(x_k) = \frac{(x_i, Ax_i)}{(x_i, x_i)}$ соотношение Релея. 

- Метод сходится **кубически для эрмитовых матриц** и квадратично в противном случае.

### Неточный метод обратной итерации

- Матрицы $(A- \sigma I)$ также как $(A-R(x_i) I)$ плохо обусловлены, если $\sigma$ или $R(x_i)$ близки к собственным значениям.

- Поэтому если у вас нет LU разложения этой матрицы, то могут возникнуть проблемы при решении систем на каждой итерации

- На практике вы можете решать системы **только с некоторой точностью**. Напомним также, что число обусловленности даёт оценку сверху, которая может быть завышенной для подходящих правых частей. Поэтому даже в методе Релея близость сдвига к собственному значению существенно [не ухудшает](http://www.sciencedirect.com/science/article/pii/S0024379505005756) сходимость итерационного метода.

- Если точность решения системы возрастает от итерации к итерации, сверхлинейная сходимость для метода Релея также присутствует, см [Theorem 2.1](http://www.sciencedirect.com/science/article/pii/S0024379505005756). Иначе вы получите только линейную сходимость.

Перед тем как мы перейдём к продвинутым методам, обсудим важную концепцию **аппроксимации Ритца**.

### Аппроксимация Ритца

- Для данного подпространства, натянутого на столбцы унитарной матрицы $Q_k$ размера $N\times k$, рассмотрим спроецированную матрицу $Q_k^* A Q_k$.

- Пусть $\Theta_k=\mathrm{diag}(\theta_1,\dots,\theta_k)$ и $S_k=\begin{bmatrix}s_1 & \dots & s_k \end{bmatrix}$ матрицы собственных значений и собственных векторов для матрицы $Q_k^* A Q_k$: 

$$(Q_k^* A Q_k)S_k = S_k \Theta_k$$

тогда $\{\theta_i\}$ называются **числами Ритца** и $y_i = Q_k s_i$ - **векторы Ритца**.

### Свойства аппроксимации Ритца

- Заметим, что числа и векторы Ритца не являются собственными значениями и собственными векторами исходной матрицы $AY_k\not= Y_k \Theta_k$, но выполнено следующее равенство:

    $$Q_k^* (AY_k - Y_k \Theta_k) = Q_k^* (AQ_k S_k - Q_k S_k \Theta_k) = 0,$$

   таким образом невязка для аппроксимации Ритца **ортогональна** подпространству, натянутому на столбцы $Q_k$.

- $\lambda_\min(A) \leq \theta_\min \leq \theta_\max \leq \lambda_\max(A)$. Действительно, используя отношение Релея:

    $$\theta_\min = \lambda_\min (Q_k^* A Q_k) = \min_{x\not=0} \frac{x^* (Q_k^* A Q_k) x}{x^* x} = \min_{y\not=0:y=Q_k x} \frac{y^*  A y}{y^* y}\geq \min_{y\not= 0} \frac{y^*  A y}{y^* y} = \lambda_\min(A).$$

- Очевидно, что $\lambda_\min (Q_k^* A Q_k) = \lambda_\min(A)$, если $k=N$, но мы хотим построить базис размера $k\ll N$ такой что $\lambda_\min (Q_k^* A Q_k) \approx \lambda_\min(A)$.

- Таким же образом можно показать, что $\theta_\max \leq \lambda_\max(A)$.

### <font color='red'>Метод Релея-Ритца</font>

Таким образом, если подпространство $V$ приближает первые $k$ собственных векторов, тогда вы можете использовать **метод Релея-Ритца**:

1. Найти ортонормированный базис $Q_k$ в $V$ (например с помощью QR разложения)
2. Вычислить $Q_k^*AQ_k$
3. Вычислить вектора и числа Ритца
4. Заметим, что также можно использовать $V$ без ортогонализации, но в этом случае нужно будет решать обобщённую задачу на собственные векторы $(V^*AV)s_i = \theta_i (V^*V)s_i$.

Вопрос в том, как найти хорошее подпространство $V$?

#### Какое подпространство мы будем использовать?

#### Метод Ланцоша и Арнольди

- Хорошим выбором $V$ будет **Крыловское подпространство**.

- Напомним, что в степенном методе мы использовали только один Крыловский вектор

$$x_k = \frac{A^k x_0}{\|A^k x_0\|}.$$

- В этом случае $\theta_k = \frac{x_k^* A x_k}{x_k^* x_k}$ есть ни что иное как число Ритца. Естественная идея – использовать Крыловское пространство большей размерности.

- В результате мы найдём больше собственных значений, а также сходимость к собственному вектору для $\lambda_\max$ будет быстрее, чем в степенном методе.

- Для эрмитовой матрицы из соотношения Арнольди следует, что 

$$ Q_k^*AQ_k = T_k, $$

где $Q_k$ ортогональный базис в Крыловском подпространстве, сгенерированный процессом Ланцоша и $T_k$ трёхдиагональная матрица.

- В соответствии с методом Релея-Ритца мы ожидаем, что собственные значения $T_k$ приближают собственные значения матрицы $A$. 
- Этот метод называется **методом Ланцоша**. 
- Для несимметричных матриц он называется **методом Арнольди** и вместо трёхдиагональной матрицы мы получим верхне-гессенбергову матрицу.

Давайте теперь покажем, что  $\theta_\max \approx\lambda_\max$.

### Почему $\theta_\max \approx \lambda_\max$?

- Обозначим $\theta_1 \equiv \theta_\max$ и $\lambda_1 \equiv \lambda_\max$. Тогда

$$ \theta_1 = \max_{y\in \mathcal{K}_i, y\not=0}\frac{(y,Ay)}{(y,y)} = \max_{p_{i-1}} \frac{(p_{i-1}(A)x_0, A p_{i-1}(A)x_0)}{(p_{i-1}(A)x_0, p_{i-1}(A)x_0)}, $$

где $p_{i-1}$ полином степени не выше $i-1$ такой что $p_{i-1}(A)x_0\not=0$.

- Разложим $x_0 = \sum_{j=1}^N c_j v_j$, где $v_j$ собственные векторы $A$ (которые образуют ортонормированный базис).

- Поскольку $\theta_1 \leq \lambda_1$ получим

$$ \lambda_1 - \theta_1 \leq \lambda_1 - \frac{(p_{i-1}(A)x_0, A p_{i-1}(A)x_0)}{(p_{i-1}(A)x_0, p_{i-1}(A)x_0)} $$

для любого полинома $p_{i-1}$. Таким образом

\begin{align*}
\lambda_1 - \theta_1 &\leq \lambda_1 - \frac{\sum_{k=1}^N \lambda_k |p_{i-1}(\lambda_k)|^2 |c_k|^2}{\sum_{k=1}^N |p_{i-1}(\lambda_k)|^2 |c_k|^2} = \frac{\sum_{k=2}^N (\lambda_1 - \lambda_k) |p_{i-1}(\lambda_k)|^2 |c_k|^2}{|p_{i-1}(\lambda_1)|^2 |c_1|^2 + \sum_{k=2}^N |p_{i-1}(\lambda_k)|^2 |c_k|^2} \\
& \leq (\lambda_1 - \lambda_n) \frac{\max_{2\leq k \leq N}|p_{i-1}(\lambda_k)|^2}{|p_{i-1}(\lambda_1)|^2 }\gamma, \quad \gamma = \frac{\sum_{k=2}^N|c_k|^2}{|c_1|^2}
\end{align*}

- Так как неравенство выполнено для любого полинома $p_{i-1}$, мы выберем полином: 

$$|p_{i-1}(\lambda_1)| \gg \max_{2\leq k \leq N}|p_{i-1}(\lambda_k)|.$$

- Это неравенство выполнено например для полиномов Чебышёва на $[\lambda_n,\lambda_2]$. 
- В итоге $\theta_1 \approx \lambda_1$ или более точно:

$$
    \lambda_1 - \theta_1 \leq \frac{\lambda_1 - \lambda_n}{T_{i-1}^2(1 + 2\mu)}\gamma, \quad \mu = \frac{\lambda_1 - \lambda_2}{\lambda_2 - \lambda_n},
$$

где $T_{i-1}$ – полином Чебышёва.

### Демо: аппроксимация максимального собственного значения с помощью метода Ланцоша

In [2]:
import scipy as sp
import scipy.sparse
from scipy.sparse import csc_matrix, csr_matrix
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.sparse.linalg
import copy
import numpy as np
n = 40
ex = np.ones(n)
lp1 = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr')
e = sp.sparse.eye(n)
A = sp.sparse.kron(lp1, e) + sp.sparse.kron(e, lp1)

def lanczos(A, m):
    n = A.shape[0]
    v = np.random.random((n, 1))
    v = v / np.linalg.norm(v)
    v_old = np.zeros((n, 1))
    beta = np.zeros(m)
    alpha = np.zeros(m)
    for j in range(m-1):
        w = A.dot(v)
        alpha[j] = w.T.dot(v)
        w = w - alpha[j] * v - beta[j] * v_old
        beta[j+1] = np.linalg.norm(w)
        v_old = v.copy()
        v = w / beta[j+1]
    w = A.dot(v)
    alpha[m-1] = w.T.dot(v)
    A = np.diag(beta[1:], k=-1) + np.diag(beta[1:], k=1) + np.diag(alpha[:], k=0)
    l, _ = np.linalg.eigh(A)
    return l

# Approximation of the largest eigenvalue for different k
l_large_exact = sp.sparse.linalg.eigsh(A, k=99, which='LM')[0][0]
print('k=10, err = {}'.format(np.abs(l_large_exact - lanczos(A, 10)[0])))
print('k=20, err = {}'.format(np.abs(l_large_exact - lanczos(A, 20)[0])))
print('k=100, err = {}'.format(np.abs(l_large_exact - lanczos(A, 100)[0])))

k=10, err = 0.14888851257280145
k=20, err = 0.030614059863341758
k=100, err = 3.6844447492967447e-09


### Устойчивость

- Векторы Ланцоша могут терять ортогональность в процессе вычисления из-за арифметики с плавающей точкой, поэтому все практически полезные реализации используют **рестарты**.

- Очень хорошее введение в тему содержится в книге **Matrix Computations** авторов G. Golub и Ch. Van-Loan.

#### Другие недостатки метода Ланцоша

- Применение метода Ланцоша напрямую к матрице $A$ может привести к очень медленной сходимости, если  $\lambda_i\approx \lambda_{i+1}$ (обычно это происходит для наименьших собственных значений, которые плохо разделены)

- Для ускорения сходимости можно применить метод Ланцоша к матрице $(A-\sigma I)^{-1}$, но в этом случае системы должны решаться **очень точно**. В противном случае соотношение Арнольди перестанет выполняться.

Альтернативой этому подходу являются так называемые предобусловленные итерационные методы, например:
1. PINVIT (Предобусловленный метод обратной итерации)
2. LOBPCG (локально оптимальный блочный предобусловленный метод сопряжённых градиентов)
3. Метод Якоби-Дэвидсона (Jacobi-Davidson method)

### PINVIT (предобусловленный метод обратной итерации)

#### Получение метода

- Рассмотрим отношение Релея $R(x) = \frac{(x,Ax)}{(x,x)}$. Тогда

$$ \nabla R(x) = \frac{2}{(x,x)} (Ax - R(x) x), $$

и простейший метод градиентного спуска с предобуславливателем $B$ записывается в виде

$$ x_{i+1} = x_{i} - \tau_i B^{-1} (Ax_i - R(x_i) x_i), $$

$$ x_{i+1} = \frac{x_{i+1}}{\|x_{i+1}\|}. $$

- Обычно $B\approx A-\sigma I$, где $\sigma$ обозначает сдвиг.

- Чем $\sigma$ ближе к необходимому собственному значению, тем быстрее сходимость.

- Параметр $\tau_k$ выбирается так, чтобы минимизировать $R(x_{i+1})$ по $\tau_k$ (метод наискорейшего спуска).

- Эта процедура минимизации может быть рассмотрена как минимизация в базисе $V = [x_i, r_i]$, где $r_{i}=B^{-1} (Ax_i - R(x_i) x_i)$.

- Это приводит к обобщённой задаче на собственные значения $(V^*AV)\begin{bmatrix}1 \\ -\tau_i \end{bmatrix} = \theta (V^*V) \begin{bmatrix}1 \\ -\tau_i \end{bmatrix}$ (процедура Релея-Ритца без ортогонализации $V$). Здесь $\theta$ – ближайшее число к требуемому собственному значению.

#### Сходимость

**Теорема** ([Knyazev и Neymeyr](http://www.sciencedirect.com/science/article/pii/S002437950100461X)) 

Пусть 
- $R(x_{i})\in [\lambda_j,\lambda_{j+1}]$
- $R(x_{i+1})\in [R(x_{i}),\lambda_{j+1}]$ (случай $R(x_{i+1})\in [\lambda_{j}, R(x_{i})]$ аналогичен)
- $\|I - B^{-1} A\|_A \leq \gamma < 1$

тогда

$$
\left|\frac{R(x_{i+1}) - \lambda_j}{R(x_{i+1}) - \lambda_{j+1}}\right| < \left[ 1 - (1-\gamma)\left(1 - \frac{\lambda_j}{\lambda_{j+1}}\right) \right]^2 \cdot \left|\frac{R(x_{i}) - \lambda_j}{R(x_{i}) - \lambda_{j+1}}\right|
$$

#### Блочный случай

- Для поиска $k$ собственных значений можно делать один шаг метода PINVIT для каждого вектора:


$$ x^{(j)}_{i+1} = x^{(j)}_{i} - \tau^{(j)}_i B^{-1} (Ax^{(j)}_i - R(x^{(j)}_i) x^{(j)}_i), \quad j=1,\dots,k $$

$$ x^{(j)}_{i+1} = \frac{x^{(j)}_{i+1}}{\|x^{(j)}_{i+1}\|}. $$

- После чего ортогонализовать их с помощью QR разложения. Однако лучше использовать процедуру Релея-Ритца:

    - Пусть $X^{i}_k = [x^{(1)}_{i},\dots, x^{(k)}_{i}]$ и $R^{i}_k = [B^{-1}r^{(1)}_{i},\dots, B^{-1}r^{(k)}_{i}]$, где $r^{(j)}_{i} = Ax^{(j)}_i - R(x^{(j)}_i) x^{(j)}_i$
    - $V = [X^{i}_k, R^{i}_k]$, используем процедуру Релея-Ритца для $V$, чтобы найти новый $X^{i+1}_k$.

## LOBPCG (локально оптимальный блочный предобусловленный метод CG)

### Локально оптимальный предобусловленный метод  СG (пока ещё не блочный)

LOPCG метод

$$ x_{i+1} = x_{i} - \alpha_i B^{-1} (Ax_i - R(x_i) x_i) + \beta_i x_{i-1} , $$

$$ x_{i+1} = \frac{x_{i+1}}{\|x_{i+1}\|} $$

превосходит метод PINVIT, поскольку он добавляет в базис не только $x_i$ и $r_i$, но также $x_{i-1}$.

Однако такая интерпретация ведёт к неустойчивому алгоритму, так как $x_{i}$ становится коллинеарен $x_{i-1}$ вместе со сходимостью метода.

### LOPCG (устойчивая версия)

- A. Knyazev предложил эквивалентную устойчивую версию, которая вводит новые векторы $p_i$ (сопряжённые градиенты)

$$ p_{i+1} = r_{i} + \beta_i p_{i}, $$

$$ x_{i+1} = x_{i} + \alpha_i p_{i+1}. $$

- Можно показать, что $\mathcal{L}(x_{i},x_{i-1},r_{i})=\mathcal{L}(x_{i},p_{i},r_{i})$.

Устойчивая версия объясняет название метода:

- В стандартном методе CG мы бы минимизировали отношение Релея $R$ по направлению сопряжённого направления $p_{i+1}$: 

$$\alpha_i = \arg\min_{\alpha_i} R(x_i + \alpha_i p_{i+1}).$$

- В локально оптимальном CG мы минимизируем по отношению к двум параметрам:  

$$\alpha_i, \beta_i = \arg\min_{\alpha_i,\beta_i} R\left(x_i + \alpha_i p_{i+1}\right) = \arg\min_{\alpha_i,\beta_i} R\left(x_i + \alpha_i (r_{i} + \beta_i p_{i})\right)$$

и получаем локально более оптимальное решение. Поэтому метод называется **локально оптимальным**.

- По аналогии с методом PINVIT коэффициенты $\alpha_i,\beta_i$ можно найти из процедуру Релея-Ритца.

### Локально оптимальный <font color='red'> блочный </font> предобусловленный метод CG

- В блочной версии по аналогии с методом PINVIT на каждой итерации дан базис $V=[X^{(i)}_k,B^{-1}R^{(i)}_k, P^{(i)}_k]$ и используется процедура Релея-Ритца.

Общая схема алгоритма:

1. Найти $\tilde A = V^* A V$
2. Найти $\tilde M = V^*V$
3. Решить обобщённую задачу на собственные значения $\tilde A S_k = \tilde M S_k \Theta_k$
4. $P^{(i+1)}_{k} = [B^{-1}R^{(i)}_k, P^{(i)}_k]S_k[:,k:]$
5. $X^{(i+1)}_{k} = X^{(i)}_k S_k[:,:k] + P^{(i+1)}_{k}$ (аналогично для $X^{(i+1)}_{k} = VS_k$)
6. Вычислить новое значение $B^{-1}R^{(i+1)}_k$
7. Задать $V=[X^{(i+1)}_k,B^{-1}R^{(i+1)}_k, P^{(i+1)}_k]$, goto 1.


- Метод также сходится линейно, но быстрее чем PINVIT.

### LOBPCG: резюме

- Локально оптимальный предобусловленный метод

- Линейная сходимость

- Предобуславливатель $A-\sigma I$ не всегда хорош для задачи на собственные значения

Следующий метод (Якоби-Дэвидсона) оснащён более "умным" предобуславливаталем и даёт сверхлинейную сходимость (если системы решаются точно)!

### Метод Якоби-Дэвидсона (JD)

Метод Якоби-Дэвидсона очень популярен для решения задачи на собственные значения (не только симметричной!).

Он состоит из двух **основных ингредиентов**:

- Для данного предобуславливателя к $A-R(x_j) I$ он автоматически строит хороший предобуславливатель для задачи на собственные значения:

$$ B = (I - x_j x^*_j) (A - R(x_j) I) (I - x_j x^*_j), $$

где $x_j$ аппроксимация собственного вектора на $j$-ой итерации

- Заметим, что приближение $(A-R(x_j) I)^{-1}$ иногда не является хорошим предобуславливателем.

- Дополнительно он прибавляет к подпространству $V$ решения с предыдущих итераций (**ускорение на подпространстве**)

### Вывод метода JD

- Метод Якоби-Дэвидсона имеет красивую интерпретацию через оптимизацию на многообразии. 
- Он является **римановым методом Ньютона** на сфере и $P = I - x_j x^*_j$ проекция на касательное пространство к сфере в точке $x_j$.

Но мы получим этот метод по аналогии с оригинальной работой.

### Уравнение коррекции Якоби

- Якоби предложил не только метод решения задачи на собственные значения через вращения, но также итерационный метод для решения этой задачи. 
- Пусть $x_j$ текущая аппроксимация собственного вектора, а $t$ коррекция к нему:

$$A(x_j + t) = \lambda (x_j + t),$$

и мы ищем коррекцию $t \perp x_j$ (новый ортогональный вектор).

- Тогда его параллельная компонента имеет вид

$$x_j x^*_j A (x_j + t) = \lambda x_j,$$

что приводит к выражению

$$R(x_j) + x^* _j A t = \lambda.$$

- Ортогональная часть  

$$( I - x_j x^*_j) A (x_j + t) = (I - x_j x^*_j) \lambda (x_j + t),$$

что эквивалентно 

$$
  (I - x_j x^*_j) (A - \lambda I) t = (I - x_j x^*_j) (- A x_j + \lambda x_j) = - (I - x_j x^*_j) A x_j = - (A - R(x_j) I) x_j = -r_j,
$$

где $r_j$ – вектор невязки.

- Так как $(I - x_j x^*_j) t  = t$, мы можем переписать это уравнение в симметричной форме

$$ (I - x_j x^*_j) (A - \lambda I) (I - x_j x^*_j) t = -r_j.$$

- Теперь заменим $\lambda$ на $R(x_j)$, и получим **уравнение коррекции Якоби**:

$$
 (I - x_j x^*_j) (A - R(x_j) I) (I - x_j x^*_j) t = -r_j.
$$

Так как $r_j \perp x_j$, это уравнение совместно, если $(A - R(x_j) I)$ невырождена.

### Решение уравнения коррекции Якоби

- Обычно уравнение Якоби решается неточно с помощью подходящего Крыловского метода.

- Даже неточное решение уравнения Якоби обеспечивает ортогональность $t$ к вектору $x_j$ (почему?), что хорошо для вычислений.

#### Связь с методом Релея

- Если уравнение решается точно, мы получим метод Релея! Давайте это покажем.

$$ (I - x_j x^*_j) (A - R(x_j) I) (I - x_j x^*_j) t = -r_j.$$

$$ (I - x_j x^*_j) (A - R(x_j) I) t = -r_j.$$

$$  (A - R(x_j) I) t - \alpha x_j = -r_j, \quad \alpha = x^*_j (A - R(x_j) I) x_j$$

$$   t = \alpha (A - R(x_j) I)^{-1}x_j  - (A - R(x_j) I)^{-1}r_j,$$

- Таким образом, так как $(A - R(x_j) I)^{-1}r_j = (A - R(x_j) I)^{-1}(A - R(x_j) I)x_j = x_j$ мы получим

$$x_{j+1} = x_j + t = \alpha (A - R(x_j) I)^{-1}x_j,$$

что совпадает с методом Релея с точностью до нормировочной постоянной.

### Предобуславливание уравнения Якоби

Популярный предобуславливатель для решения уравнения Якоби имеет вид 

$$
\widetilde K = (I - x_j x^*_j) K (I - x_j x^*_j)
$$

где $K$ легко обратимая аппроксимация $A - R(x_j) I $.

- Нам нужно получить метод решения системы с $\widetilde K$ в терминах решения системы с $K$.

- Мы уже видели, что уравнение

$$ (I - x_j x^*_j) K (I - x_j x^*_j) \tilde t = f $$

эквивалентно

$$  \tilde t = \alpha K^{-1}x_j  + K^{-1}f $$

- А сейчас мы забудем о значении $\alpha$ и будем искать его из требования $\tilde t\perp x_j$ для поддержки ортогональности:

$$ \alpha = \frac{x_j^*K^{-1}f}{x_j^* K^{-1}x_j} $$

- Таким образом, для каждой итерации решения уравнения Якоби мы вычислим $K^{-1}x_j$ после чего обновим только $K^{-1}f$ на каждой внутренней итерации Крыловского метода

#### Ускорение на подпространстве в JD

- На каждой итерации метода мы расширим базис с помощью нового $t$.

- А именно $V_j = [v_1,\dots,v_{j-1},v_j]$, где $v_j$ – вектор $t$, ортогонализованный к $V_{j-1}$.

- Затем используется стандартная процедура Релея-Ритца

**Исторический факт:** сначала ускорение на подпространстве использовалось в методе Дэвидсона

- В этом методе вместо уравнения Якоби решалось уравнение $(\mathrm{diag}(A) - R(x_j)I)t = -r_j$
- Метод Дэвидсона был очень популярен при решении задач квантовой химии.

#### Блочный случай в методе JD

Если мы хотим найти несколько собственных векторов, мы вычислим **частичное разложение Шура:**

$$A Q_k = Q_k T_k, $$

и тогда хотим обновить $Q_k$ с помощью одного вектора, добавленного к $Q_k$. Мы будем использовать вместо матрицы $A$ матрицу $(I - Q_k Q^*_k) A (I - Q_k Q^*_k)$.

### JD: итог

- Уравнение коррекции может быть решено неточно и метод JD часто самый быстрый.

### Реализации

- [ARPack](http://www.caam.rice.edu/software/ARPACK/) наиболее широко используемый пакет для решения частичной задачи на собственные значения (он также запускается внутри SciPy). Содержит варианты алгоритмов Ланцоша и Арнольди
- [PRIMME](https://github.com/primme/primme) – лучший по нашему опыту (использует динамическое переключение между различными методами, включая LOBPCG и JD)
- [PROPACK](http://sun.stanford.edu/~rmunk/PROPACK/) работает хорошо для вычисления SVD.

### Резюме

- Методы Арнольди и Ланцоша. Shift-and-invert стратегия очень затратна, поскольку надо очень точно решать систему на каждой итерации.
- Итерационные методы с предобуславливателем (PINVIT, LOBPCG, JD) подходят для случая неточного решения системы. 
- Есть готовые пакеты для их использования
- Большое количество технических сложностей осталось за кадром (рестарты, устойчивость)

In [41]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()